# 第7章 基于RNN生成文本

seq2seq 是“(from) sequence to sequence”（从时序到时序）的意思，即将一个时序数
据转换为另一个时序数据。

通过组合两个 RNN，可以轻松实现 seq2seq。



## 7.1 使用语言模型生成文本

### 7.1.1 使用RNN生成文本的步骤

![](../images/图7-1.上一章实现的语言模型：右图使用整体处理时序数据的Time层；左图是将其展.PNG)
图7-1.上一章实现的语言模型：右图使用整体处理时序数据的Time层；左图是将其展开后的层结构

![](../images/图7-2.语言模型输出下一个出现的单词的概率分布.PNG)
图7-2.语言模型输出下一个出现的单词的概率分布

如何生成下一个新单词呢？
* 概率最高的单词
* “概率性地”进行选择

![](../images/图7-3.根据概率分布采样一个单词.PNG)
图7-3.根据概率分布采样一个单词

“确定性的”是指（算法的）结果是唯一确定的，是可预测的。
“概率性的”算法则概率性地确定结果。

![](../images/图7-4.重复概率分布的输出和采样.PNG)
图7-4.重复概率分布的输出和采样


### 7.1.2 文本生成的实现

文本生成的实现。这里基于上一章实现的 Rnnlm 类（[ch06/rnnlm.py](../ch06/rnnlm.py)），来创建继承自它的 RnnlmGen 类，
然后向这个类添加生成文本的方法。

RnnlmGen 类的实现如下所示（[ch07/rnnlm_gen.py](../ch07/rnnlm_gen.py)）

使用这个 RnnlmGen 类进行文本生成。这里先在完全没有学习的状态（即权重参数是随机初始值的状态）下生成文本，
代码如下所示（[ch07/generate_text.py](../ch07/generate_text.py)）

### 7.1.3 更好的文本生成

（[ch07/generate_better_text.py](../ch07/generate_better_text.py)）

## 7.2 seq2seq模型

文本数据、音频数据和视频数据都是时序数据。
将一种时序数据转换为另一种时序数据的任务，比如机器翻译、语音识别等。

### 7.2.1 seq2seq的原理

seq2seq 模型也称为 Encoder-Decoder 模型。顾名思义，这个模型有两
个模块——Encoder（编码器）和 Decoder（解码器）。编码器对输入数据
进行编码，解码器对被编码的数据进行解码。

![](../images/图7-5.基于编码器和解码器进行翻译的例子.PNG)
图7-5.基于编码器和解码器进行翻译的例子

编码器首先对“吾輩は猫である”这句话进行编码，然后将编码好的信息传递给解码器，由解码器生成目标文本。
此时，编码器编码的信息浓缩了翻译所必需的信息，解码器基于这个浓缩的信息生成目标文本。

![](../images/图7-6.编码器的层结构.PNG)
图7-6.编码器的层结构

![](../images/图7-7.编码器将文本编码为固定长度的向量.PNG)
图7-7.编码器将文本编码为固定长度的向量

![](../images/图7-8.解码器的层结构.PNG)
图7-8.解码器的层结构

![](../images/图7-9.seq2seq的整体的层结构.PNG)
图7-9.seq2seq的整体的层结构

### 7.2.2 时序数据转换的简单尝试

![](../images/图7-10.让seq2seq学习加法的例子.PNG)
图7-10.让seq2seq学习加法的例子

### 7.2.3 可变长的时序数据

在使用批数据进行学习时，会一起处理多个样本。此时，（在我们的
实现中）需要保证一个批次内各个样本的数据形状是一致的。

在基于 mini-batch 学习可变长度的时序数据时，最简单的方法是使用
**填充**（padding）。所谓填充，就是用无效（无意义）数据填入原始数据，从
而使数据长度对齐。

![](../images/图7-11.为了进行mini-batch学习，使用空白字符进行填充，使输入和输出的大小对齐.PNG)
图7-11.为了进行mini-batch学习，使用空白字符进行填充，使输入和输出的大小对齐

### 7.2.4 加法数据集

![](../images/图7-12.加法的学习数据：空白字符（空格）用灰色的点表示.PNG)
图7-12.加法的学习数据：空白字符（空格）用灰色的点表示

专用模块（[dataset/sequence.py](../dataset/sequence.py)），这个模块有 load_data() 和 get_vocab() 两个方法

load_data(file_name, seed) 读入由 file_name 指定的文本文件，并将文
本转换为字符 ID，返回训练数据和测试数据。该方法内部设有随机数种子
seed 以打乱数据，分割训练数据和测试数据。另外，get_vocab() 方法返回
字符与 ID 的映射字典（实际上返回 char_to_id 和 id_to_char）。

（[ch07/show_addition_dataset.py](../ch07/show_addition_dataset.py)）。

## 7.3 seq2seq的实现

seq2seq 是组合了两个 RNN 的神经网络。这里我们首先将这两个 RNN
实现为 Encoder 类和 Decoder 类，然后将这两个类组合起来，来实现 seq2seq 类。

### 7.3.1 Encoder类

![](../images/图7-13.Encoder类的输入输出.PNG)
图7-13.Encoder类的输入输出

![](../images/图7-14.编码器的层结构.PNG)
图7-14.编码器的层结构

如图 7-14 所示，Encoder 类由 Embedding 层和 LSTM 层组成。

在编码器处理完最后一个字符后，输出 LSTM 层的隐藏状态 h。然后，这个隐藏状态 h 被传递给解码器。

![](../images/图7-15.使用Time层实现编码器.PNG)
图7-15.使用Time层实现编码器

### 7.3.2 Decoder类

Decoder 类的实现。如图 7-16 所示，Decoder 类接收 Encoder 类输出的 h，输出目标字符串。

![](../images/图7-16.编码器和解码器.PNG)
图7-16.编码器和解码器

![](../images/图7-17.解码器的层结构（学习时）.PNG)
图7-17.解码器的层结构（学习时）

![](../images/图7-18.解码器生成字符串的步骤：通过argmax节点从Affine层的输出中选择最大值的索引（字符ID）.PNG)
图7-18.解码器生成字符串的步骤：通过argmax节点从Affine层的输出中选择最大值的索引（字符ID）

![](../images/图7-19.Decoder类的结构.PNG)
图7-19.Decoder类的结构

由图 7-19 可以看出，Decoder 类由 Time Embedding、Time LSTM 和
Time Affine 这 3 个层构成。

### 7.3.3 Seq2seq类

（[ch07/seq2seq.py](../ch07/seq2seq.py)）

### 7.3.4 seq2seq的评价

基础神经网络的学习流程如下：
1. 从训练数据中选择一个 mini-batch
2. 基于 mini-batch 计算梯度
3. 使用梯度更新权重

seq2seq 的学习代码（[ch07/train_seq2seq.py](../ch07/train_seq2seq.py)）

![](../images/图7-22.正确率的变化.PNG)
图7-22.正确率的变化

## 7.4 seq2seq的改进

### 7.4.1 反转输入数据（Reverse）

![](../images/图7-23.反转输入数据的例子.PNG)
图7-23.反转输入数据的例子

```python
# 读入数据集
(x_train, t_train), (x_test, t_test) = sequence.load_data('addition.txt')
# 反转数组
x_train, x_test = x_train[:, ::-1], x_test[:, ::-1]
```

![](../images/图7-24.seq2seq的正确率的变化：baseline是上一节的结果，reverse是反转输入数据后的结果.PNG)
图7-24.seq2seq的正确率的变化：baseline是上一节的结果，reverse是反转输入数据后的结果

为什么反转数据后，学习进展变快，精度提高了呢？虽然理论上不是很清楚，但是直观上可以认为，反转数据后梯度的传播可以更平滑。

在反转输入数据后，单词之间的“平均”距离并不会发生改变。

### 7.4.2 偷窥（Peeky）

编码器将输入语句转换为固定长度的向量 h，这个 h集中了解码器所需的全部信息。也就是说，它是解码器唯一的信息源。

![](../images/图7-25.改进前：只有最开始的LSTM层接收编码器的输出h.PNG)
图7-25.改进前：只有最开始的LSTM层接收编码器的输出h

将这个集中了重要信息的编码器的输出 h 分配给解码器的其他层。

![](../images/图7-26.改进后：将编码器的输出h分配给所有时刻的LSTM层和Affine层.PNG)
图7-26.改进后：将编码器的输出h分配给所有时刻的LSTM层和Affine层

有两个向量同时被输入到了 LSTM 层和 Affine 层，这实际上表示两个向量的拼接（concatenate）。

![](../images/图7-27.在Affine层的输入有两个的情况下（左图），将它们拼接起来输入Affine层（右图）.PNG)
图7-27.在Affine层的输入有两个的情况下（左图），将它们拼接起来输入Affine层（右图）

PeekyDecoder 类的实现[ch07/peeky_seq2seq.py](../ch07/peeky_seq2seq.py)

![](../images/图7-28.“reverse+peeky”是进行了本节的两个改进的结.PNG)
图7-28.“reverse+peeky”是进行了本节的两个改进的结

使用 Peeky 后，网络的权重
参数会额外地增加，计算量也会增加，所以这里的实验结果必须考虑到相应
地增加的“负担”。另外，seq2seq 的精度会随着超参数的调整而大幅变化。
虽然这里的结果是可靠的，但是在实际问题中，它的效果可能不稳定。

## 7.5 seq2seq的应用

* 机器翻译：将“一种语言的文本”转换为“另一种语言的文本”
* 自动摘要：将“一个长文本”转换为“短摘要”
* 问答系统：将“问题”转换为“答案”
* 邮件自动回复：将“接收到的邮件文本”转换为“回复文本”

### 7.5.1 聊天机器人

### 7.5.2 算法学习

### 7.5.3 自动图像描述

将图像转换为文本的**自动图像描述**（image captioning）

![](../images/图7-31.用于自动图像描述的seq2seq的网络结构示例.PNG)
图7-31.用于自动图像描述的seq2seq的网络结构示例

它和之前的网络的唯一区别在于，编码器从 LSTM 换成了 CNN（Convolutional Neural Network，卷积神经网络），而解码器仍使用与之前相同的网络。仅通过这点改变（用
CNN 替代 LSTM），seq2seq 就可以处理图像了。

图 7-31 的 CNN 使用 VGG、ResNet 等成熟网络，并使用在别的图
像数据集（ImageNet 等）上学习好的权重，这样可以获得好的编码，
从而生成好的文本。

![](../images/图7-32.自动图像描述的例子：将图像转换为文本.PNG)
图7-32.自动图像描述的例子：将图像转换为文本

## 7.6 小结

seq2seq 模型拼接了编码器和解码器，是组合了两个 RNN 的简单结构。
改进 seq2seq 的两个方案——Reverse 和 Peeky。

## 本章所学的内容

* 基于 RNN 的语言模型可以生成新的文本
* 在进行文本生成时，重复“输入一个单词（字符），基于模型的输出（概率分布）进行采样”这一过程
* 通过组合两个 RNN，可以将一个时序数据转换为另一个时序数据（seq2seq）
* 在 seq2seq 中，编码器对输入语句进行编码，解码器接收并解码这个编码信息，获得目标输出语句
* 反转输入语句（Reverse）和将编码信息分配给解码器的多个层（Peeky）可以有效提高 seq2seq 的精度
* seq2seq 可以用在机器翻译、聊天机器人和自动图像描述等各种各样的应用中

